In [1]:
import sys
import pandas as pd
import numpy as np
from typing import Any
from scipy.stats import linregress, ttest_1samp

%load_ext autoreload
%autoreload 2
# sys.path.append("D:/Github/note/module")                        # for windows
sys.path.append("/Users/xinc./Documents/GitHub/note/module")    # for mac
from get_info_FinMind import GetInfoFinMind
FinMind_fetcher = GetInfoFinMind()
from plot_func import plot, plot_scatter, plot_df_columns, plot_pdf, plot_dropped_positions, plot_sequence
from performance_func import summarize_performance, mean_ttest

start_date = pd.to_datetime("2020-01-01", format = "%Y-%m-%d")
end_date = pd.to_datetime("2025-09-25", format = "%Y-%m-%d")


# download data

In [157]:
# TXO
op_df = FinMind_fetcher.get_option_daily("TXO", start_date, end_date, trading_session = "all")
op_df.reset_index(drop = True, inplace = True)
op_df_position = op_df[op_df["trading_session"] == "position"].copy()

# TXOa
op_df_after = op_df[op_df["trading_session"] == "after_market"].copy()
op_df_position.to_pickle("../data/TXO.pkl")
op_df_after.to_pickle("../data/TXO_after.pkl")

# TX
tx_df = FinMind_fetcher.init_df("TX", start_date, end_date)
tx_df = FinMind_fetcher.get_future_price()
tx_df.to_csv("../data/TX.csv")

# TXa
txa_df = FinMind_fetcher.init_df("TX", start_date, end_date)
txa_df = FinMind_fetcher.get_future_price(trading_session = "after_market")
txa_df.to_csv("../data/TX_after.csv")


2025-10-21 11:10:47.150 | INFO     | FinMind.data.finmind_api:get_data:158 - download Dataset.TaiwanOptionDaily, data_id: TXO
2025-10-21 11:12:26.065 | INFO     | FinMind.data.finmind_api:get_data:158 - download Dataset.TaiwanFuturesDaily, data_id: TX
/Users/xinc./Documents/GitHub/note/module/get_info_FinMind.py:267: FutureWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.

2025-10-21 11:12:28.603 | INFO     | FinMind.data.finmind_api:get_data:158 - download Dataset.TaiwanFuturesDaily, data_id: TX
/Users/xinc./Documents/GitHub/note/module/get_info_FinMind.py:267: FutureWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping col

# read

In [24]:
"""read tx, txo"""
tx_df = pd.read_csv("../data/TX.csv")
txa_df = pd.read_csv("../data/TX_after.csv")
op_df = pd.read_pickle("../data/TXO.pkl")
opa_df = pd.read_pickle("../data/TXO_after.pkl")
twii_df = pd.read_pickle("../data/twii.pkl")
twii_df.index = pd.to_datetime(twii_df.index)
twii_df = twii_df.loc[start_date: end_date]

for frame in (tx_df, txa_df):
    frame["Timestamp"] = pd.to_datetime(frame["Timestamp"])
    frame["daily_ret"] = (frame["Close"] / frame["Open"]) - 1
op_df["date"] = pd.to_datetime(op_df["date"])
opa_df["date"] = pd.to_datetime(opa_df["date"])

def build_atm(option_df: pd.DataFrame, price_map: pd.Series, label: str) -> pd.DataFrame:
    atm = option_df.copy()
    atm[label] = atm["date"].map(price_map)
    atm = atm[atm[label].notna()].copy()
    atm["moneyness_gap"] = (atm["strike_price"] - atm[label]).abs()
    atm = (atm.sort_values(["date", "call_put", "moneyness_gap", "volume", "open_interest"],
                          ascending = [True, True, True, False, False])
              .groupby(["date", "call_put"], as_index = False)
              .head(1)
              .reset_index(drop = True))
    return atm.drop(columns = [label, "moneyness_gap"])

_tx_close_map = tx_df.set_index("Timestamp")["Close"]
_tx_open_map = tx_df.set_index("Timestamp")["Open"]
op_atm_close_df = build_atm(op_df, _tx_close_map, "day_close")
op_atm_open_df = build_atm(op_df, _tx_open_map, "day_open")

_txa_close_map = txa_df.set_index("Timestamp")["Close"]
_txa_open_map = txa_df.set_index("Timestamp")["Open"]
opa_atm_close_df = build_atm(opa_df, _txa_close_map, "night_close")
opa_atm_open_df = build_atm(opa_df, _txa_open_map, "night_open")

# diff scenario

## 夜盤 close - 日盤 open

### put

#### ret

In [133]:
put_df = pd.DataFrame(index = opa_atm_close_df.loc[opa_atm_close_df["call_put"] == "put", "date"].unique())

# get cross ret
put_df = put_df.merge(
    opa_atm_close_df.loc[opa_atm_close_df["call_put"] == "put", ["date", "strike_price", "close"]].rename(columns = {"close": "prev_night_close"}),
    left_index = True,
    right_on = "date",
    how = "left"
)
put_df["date"] = pd.to_datetime(put_df["date"])
put_df = put_df.merge(
    op_df.loc[op_df["call_put"] == "put", ["date", "strike_price", "open"]],
    on = ["date", "strike_price"],
    how = "left"
)
put_mask = (put_df["prev_night_close"] > 0) & (put_df["open"] > 0)
put_df["cross_ret"] = np.where(put_mask, (put_df["open"] / put_df["prev_night_close"]) - 1, np.nan)

# calculate date delta
put_df["gap_days"] = put_df["date"].diff().dt.days

# get tx
merge_df = pd.merge(
    tx_df[["Timestamp", "Open"]], txa_df[["Timestamp", "Close"]],
    on = "Timestamp"
)
merge_df["tx_ret"] = (merge_df["Open"] / merge_df["Close"]) - 1
put_df["date"], merge_df["Timestamp"] = pd.to_datetime(put_df["date"]), pd.to_datetime(merge_df["Timestamp"])

put_df = put_df.merge(
    merge_df[["Timestamp", "tx_ret"]].rename(columns = {"Timestamp": "date"}),
    on = "date",
    how = "left"
)
put_df.dropna(subset = ["cross_ret", "tx_ret"], inplace = True)

In [134]:
put_df.sort_values(["gap_days", "date"], ignore_index = True, inplace = True)

# cum cross ret
mean = put_df["cross_ret"].mean()
put_df["demeaned_cross_ret"] = put_df["cross_ret"] - mean
put_df["cum_demeaned_cross_ret"] = put_df["demeaned_cross_ret"].cumsum()

# cum tx ret
mean = put_df["tx_ret"].mean()
put_df["demeaned_tx_ret"] = put_df["tx_ret"] - mean
put_df["cum_demeaned_tx_ret"] = put_df["demeaned_tx_ret"].cumsum()

# normalize cum ret
put_df[["norm_cum_demeaned_cross_ret", "norm_cum_demeaned_tx_ret"]] = pd.concat([
    put_df["cum_demeaned_cross_ret"] / put_df["cum_demeaned_cross_ret"].std(),
    put_df["cum_demeaned_tx_ret"] / put_df["cum_demeaned_tx_ret"].std()
], axis = 1)

plot(put_df.loc[put_df["gap_days"] < 2], ["norm_cum_demeaned_cross_ret", "norm_cum_demeaned_tx_ret"], ry = "gap_days")
plot(put_df.loc[put_df["gap_days"] >= 2], ["norm_cum_demeaned_cross_ret", "norm_cum_demeaned_tx_ret"], ry = "gap_days")

#### point

In [107]:
put_df = pd.DataFrame(index = opa_atm_close_df.loc[opa_atm_close_df["call_put"] == "put", "date"].unique())

# get cross ret
put_df = put_df.merge(
    opa_atm_close_df.loc[opa_atm_close_df["call_put"] == "put", ["date", "strike_price", "close"]].rename(columns = {"close": "prev_night_close"}),
    left_index = True,
    right_on = "date",
    how = "left"
)
put_df["date"] = pd.to_datetime(put_df["date"])
put_df = put_df.merge(
    op_df.loc[op_df["call_put"] == "put", ["date", "strike_price", "open"]],
    on = ["date", "strike_price"],
    how = "left"
)
put_mask = (put_df["prev_night_close"] > 0) & (put_df["open"] > 0)
put_df["cross_ret"] = np.where(put_mask, (put_df["open"] - put_df["prev_night_close"]), np.nan)

# calculate date delta
put_df["gap_days"] = put_df["date"].diff().dt.days

# get tx
merge_df = pd.merge(
    tx_df[["Timestamp", "Open"]], txa_df[["Timestamp", "Close"]],
    on = "Timestamp"
)
merge_df["tx_ret"] = (merge_df["Open"] - merge_df["Close"])
put_df["date"], merge_df["Timestamp"] = pd.to_datetime(put_df["date"]), pd.to_datetime(merge_df["Timestamp"])

put_df = put_df.merge(
    merge_df[["Timestamp", "tx_ret"]].rename(columns = {"Timestamp": "date"}),
    on = "date",
    how = "left"
)
put_df.dropna(subset = ["cross_ret", "tx_ret"], inplace = True)

In [108]:
put_df.sort_values(["gap_days", "date"], ignore_index = True, inplace = True)
mask = put_df["gap_days"] >= 2

# cum cross ret
put_df["cum_cross_ret"] = put_df["cross_ret"].cumsum()
put_df.loc[mask, "cum_cross_ret"] = put_df.loc[mask, "cross_ret"].cumsum()
mean = put_df["cross_ret"].mean()
put_df["demeaned_cross_ret"] = put_df["cross_ret"] - mean
put_df["cum_demeaned_cross_ret"] = put_df["demeaned_cross_ret"].cumsum()
put_df.loc[mask, "cum_demeaned_cross_ret"] = put_df.loc[mask, "demeaned_cross_ret"].cumsum()

# cum tx ret
put_df["cum_tx_ret"] = put_df["tx_ret"].cumsum()
put_df.loc[mask, "cum_tx_ret"] = put_df.loc[mask, "tx_ret"].cumsum()
mean = put_df["tx_ret"].mean()
put_df["demeaned_tx_ret"] = put_df["tx_ret"] - mean
put_df["cum_demeaned_tx_ret"] = put_df["demeaned_tx_ret"].cumsum()
put_df.loc[mask, "cum_demeaned_tx_ret"] = put_df.loc[mask, "demeaned_tx_ret"].cumsum()

plot(put_df.loc[put_df["gap_days"] < 2], ["cum_cross_ret", "cum_tx_ret"], ry = "gap_days")
plot(put_df.loc[put_df["gap_days"] >= 2], ["cum_cross_ret", "cum_tx_ret"], ry = "gap_days")

### call

#### ret

In [131]:
call_df = pd.DataFrame(index = opa_atm_close_df.loc[opa_atm_close_df["call_put"] == "call", "date"].unique())

call_df = call_df.merge(
    opa_atm_close_df.loc[opa_atm_close_df["call_put"] == "call", ["date", "strike_price", "close"]]
              .rename(columns = {"close": "prev_night_close"}),
    left_index = True,
    right_on = "date",
    how = "left"
)
call_df["date"] = pd.to_datetime(call_df["date"])
call_df = call_df.merge(
    op_df.loc[op_df["call_put"] == "call", ["date", "strike_price", "open"]],
    on = ["date", "strike_price"],
    how = "left"
)
call_mask = (call_df["prev_night_close"] > 0) & (call_df["open"] > 0)
call_df["cross_ret"] = np.where(call_mask, (call_df["open"] / call_df["prev_night_close"]) - 1, np.nan)
call_df["gap_days"] = call_df["date"].diff().dt.days

# get tx ret
merge_df = pd.merge(
    tx_df[["Timestamp", "Open"]], txa_df[["Timestamp", "Close"]],
    on = "Timestamp"
)
merge_df["tx_ret"] = (merge_df["Open"] / merge_df["Close"]) - 1
call_df["date"], merge_df["Timestamp"] = pd.to_datetime(call_df["date"]), pd.to_datetime(merge_df["Timestamp"])

call_df = call_df.merge(
    merge_df[["Timestamp", "tx_ret"]].rename(columns = {"Timestamp": "date"}),
    on = "date",
    how = "left"
)
call_df.dropna(subset = ["cross_ret", "tx_ret"], inplace = True)

In [132]:
call_df.sort_values(["gap_days", "date"], ignore_index = True, inplace = True)

# cum cross ret
mean = call_df["cross_ret"].mean()
call_df["demeaned_cross_ret"] = call_df["cross_ret"] - mean
call_df["cum_demeaned_cross_ret"] = call_df["demeaned_cross_ret"].cumsum()

# cum tx ret
mean = call_df["tx_ret"].mean()
call_df["demeaned_tx_ret"] = call_df["tx_ret"] - mean
call_df["cum_demeaned_tx_ret"] = call_df["demeaned_tx_ret"].cumsum()

# normalize
call_df[["norm_cum_demeaned_cross_ret", "norm_cum_demeaned_tx_ret"]] = pd.concat([
    call_df["cum_demeaned_cross_ret"] / call_df["cum_demeaned_cross_ret"].std(),
    call_df["cum_demeaned_tx_ret"] / call_df["cum_demeaned_tx_ret"].std()
], axis = 1)

plot(call_df.loc[call_df["gap_days"] < 2], ["norm_cum_demeaned_cross_ret", "norm_cum_demeaned_tx_ret"], ry = "gap_days")
plot(call_df.loc[call_df["gap_days"] >= 2], ["norm_cum_demeaned_cross_ret", "norm_cum_demeaned_tx_ret"], ry = "gap_days")

#### point

In [85]:
call_df = pd.DataFrame(index = opa_atm_close_df.loc[opa_atm_close_df["call_put"] == "call", "date"].unique())

# get cross ret
call_df = call_df.merge(
    opa_atm_close_df.loc[opa_atm_close_df["call_put"] == "call", ["date", "strike_price", "close"]]
            .rename(columns = {"close": "prev_night_close"}),
            left_index = True,
            right_on = "date",
            how = "left"
)
call_df["date"] = pd.to_datetime(call_df["date"])
call_df = call_df.merge(
    op_df.loc[op_df["call_put"] == "call", ["date", "strike_price", "open"]],
    on = ["date", "strike_price"],
    how = "left"
)
call_mask = (call_df["prev_night_close"] > 0) & (call_df["open"] > 0)
call_df["cross_ret"] = np.where(call_mask, (call_df["open"] - call_df["prev_night_close"]), np.nan)
call_df["gap_days"] = call_df["date"].diff().dt.days

# get tx ret
merge_df = pd.merge(
    tx_df[["Timestamp", "Open"]], txa_df[["Timestamp", "Close"]],
    on = "Timestamp"
)
merge_df["tx_ret"] = (merge_df["Open"] - merge_df["Close"])
call_df["date"], merge_df["Timestamp"] = pd.to_datetime(call_df["date"]), pd.to_datetime(merge_df["Timestamp"])

call_df = call_df.merge(
    merge_df[["Timestamp", "tx_ret"]].rename(columns = {"Timestamp": "date"}),
    on = "date",
    how = "left"
)
call_df.dropna(subset = ["cross_ret", "tx_ret"], inplace = True)

In [86]:
call_df.sort_values(["gap_days", "date"], ignore_index = True, inplace = True)
mask = call_df["gap_days"] >= 2

# cum cross ret
call_df["cum_cross_ret"] = call_df["cross_ret"].cumsum()
call_df.loc[mask, "cum_cross_ret"] = call_df.loc[mask, "cross_ret"].cumsum()
mean = call_df["cross_ret"].mean()
call_df["demeaned_cross_ret"] = call_df["cross_ret"] - mean
call_df["cum_demeaned_cross_ret"] = call_df["demeaned_cross_ret"].cumsum()
call_df.loc[mask, "cum_demeaned_cross_ret"] = call_df.loc[mask, "demeaned_cross_ret"].cumsum()

# cum tx ret
call_df["cum_tx_ret"] = call_df["tx_ret"].cumsum()
call_df.loc[mask, "cum_tx_ret"] = call_df.loc[mask, "tx_ret"].cumsum()
mean = call_df["tx_ret"].mean()
call_df["demeaned_tx_ret"] = call_df["tx_ret"] - mean
call_df["cum_demeaned_tx_ret"] = call_df["demeaned_tx_ret"].cumsum()
call_df.loc[mask, "cum_demeaned_tx_ret"] = call_df.loc[mask, "demeaned_tx_ret"].cumsum()

plot(call_df.loc[call_df["gap_days"] < 2], ["cum_cross_ret", "cum_tx_ret"], ry = "gap_days")
plot(call_df.loc[call_df["gap_days"] >= 2], ["cum_cross_ret", "cum_tx_ret"], ry = "gap_days")
plot(call_df.loc[call_df["gap_days"] < 2], ["cum_demeaned_cross_ret", "cum_demeaned_tx_ret"], ry = "gap_days")
plot(call_df.loc[call_df["gap_days"] >= 2], ["cum_demeaned_cross_ret", "cum_demeaned_tx_ret"], ry = "gap_days")

## 夜盤 open - 日盤 open

### put

#### ret

In [129]:
put_df = pd.DataFrame(index = opa_atm_open_df.loc[opa_atm_open_df["call_put"] == "put", "date"].unique())

# get cross ret
put_df = put_df.merge(
    opa_atm_open_df.loc[opa_atm_open_df["call_put"] == "put", ["date", "strike_price", "open"]]
              .rename(columns = {"open": "prev_night_open"}),
    left_index = True,
    right_on = "date",
    how = "left"
)

put_df["date"] = pd.to_datetime(put_df["date"])
position_open = (
    op_df.loc[(op_df["call_put"] == "put"),
              ["date", "strike_price", "open"]]
        .drop_duplicates(["date", "strike_price"])
        .rename(columns = {"open": "day_open"})
)
put_df = put_df.merge(position_open, on = ["date", "strike_price"], how = "left")
mask = (put_df["prev_night_open"] > 0) & (put_df["day_open"] > 0)
put_df["cross_ret"] = np.where(mask, (put_df["day_open"] / put_df["prev_night_open"]) - 1, np.nan)

# get date delta
put_df["gap_days"] = put_df["date"].diff().dt.days

# get tx ret
merge_df = pd.merge(
    tx_df[["Timestamp", "Open"]].rename(columns = {"Open": "position_open"}), txa_df[["Timestamp", "Open"]].rename(columns = {"Open": "after_open"}),
    on = "Timestamp"
)
merge_df["tx_ret"] = (merge_df["position_open"] / merge_df["after_open"]) - 1
put_df["date"], merge_df["Timestamp"] = pd.to_datetime(put_df["date"]), pd.to_datetime(merge_df["Timestamp"])
put_df = put_df.merge(
    merge_df[["Timestamp", "tx_ret"]].rename(columns = {"Timestamp": "date"}),
    on = "date",
    how = "left"
)
put_df.dropna(subset = ["cross_ret", "tx_ret"], inplace = True)

In [130]:
put_df.sort_values(["gap_days", "date"], ignore_index = True, inplace = True)

# cum cross_ret ret
mean = put_df["cross_ret"].mean()
put_df["demeaned_cross_ret"] = put_df["cross_ret"] - mean
put_df["cum_demeaned_cross_ret"] = put_df["demeaned_cross_ret"].cumsum()

# cum tx ret
mean = put_df["tx_ret"].mean()
put_df["demeaned_tx_ret"] = put_df["tx_ret"] - mean
put_df["cum_demeaned_tx_ret"] = put_df["demeaned_tx_ret"].cumsum()

# normalize
put_df[["norm_cum_demeaned_cross_ret", "norm_cum_demeaned_tx_ret"]] = pd.concat([
    put_df["cum_demeaned_cross_ret"] / put_df["cum_demeaned_cross_ret"].std(),
    put_df["cum_demeaned_tx_ret"] / put_df["cum_demeaned_tx_ret"].std()
], axis = 1)

plot(put_df.loc[put_df["gap_days"] < 2], ["norm_cum_demeaned_cross_ret", "norm_cum_demeaned_tx_ret"], ry = "gap_days")
plot(put_df.loc[put_df["gap_days"] >= 2], ["norm_cum_demeaned_cross_ret", "norm_cum_demeaned_tx_ret"], ry = "gap_days")

#### point

In [115]:
put_df = pd.DataFrame(index = opa_atm_open_df.loc[opa_atm_open_df["call_put"] == "put", "date"].unique())

# get cross ret
put_df = put_df.merge(
    opa_atm_open_df.loc[opa_atm_open_df["call_put"] == "put", ["date", "strike_price", "open"]]
                .rename(columns = {"open": "prev_night_open"}),
    left_index = True,
    right_on = "date",
    how = "left"
)

put_df["date"] = pd.to_datetime(put_df["date"])
position_open = (
    op_df.loc[op_df["call_put"] == "put", ["date", "strike_price", "open"]]
    .drop_duplicates(["date", "strike_price"])
    .rename(columns = {"open": "day_open"})
)
put_df = put_df.merge(position_open, on = ["date", "strike_price"], how = "left")
mask = (put_df["prev_night_open"] > 0) & (put_df["day_open"] > 0)
put_df["cross_ret"] = np.where(mask, (put_df["day_open"] - put_df["prev_night_open"]), np.nan)

# get date delta
put_df["gap_days"] = put_df["date"].diff().dt.days

# get tx ret
merge_df = pd.merge(
    tx_df[["Timestamp", "Open"]].rename(columns = {"Open": "position_open"}), txa_df[["Timestamp", "Open"]].rename(columns = {"Open": "after_open"}),
    on = "Timestamp"
)
merge_df["tx_ret"] = (merge_df["position_open"] - merge_df["after_open"])
put_df["date"], merge_df["Timestamp"] = pd.to_datetime(put_df["date"]), pd.to_datetime(merge_df["Timestamp"])
put_df = put_df.merge(
    merge_df[["Timestamp", "tx_ret"]].rename(columns = {"Timestamp": "date"}),
    on = "date",
    how = "left"
)
put_df.dropna(subset = ["cross_ret", "tx_ret"], inplace = True)

In [116]:
put_df.sort_values(["gap_days", "date"], ignore_index = True, inplace = True)
mask = put_df["gap_days"] >= 2

# cum cross_ret ret
put_df["cum_cross_ret"] = put_df["cross_ret"].cumsum()
put_df.loc[mask, "cum_cross_ret"] = put_df.loc[mask, "cross_ret"].cumsum()
mean = put_df["cross_ret"].mean()

# cum tx ret
put_df["cum_tx_ret"] = put_df["tx_ret"].cumsum()
put_df.loc[mask, "cum_tx_ret"] = put_df.loc[mask, "tx_ret"].cumsum()
mean = put_df["tx_ret"].mean()

plot(put_df.loc[put_df["gap_days"] < 2], ["cum_cross_ret", "cum_tx_ret"], ry = "gap_days")
plot(put_df.loc[put_df["gap_days"] >= 2], ["cum_cross_ret", "cum_tx_ret"], ry = "gap_days")

### call

#### ret

In [127]:
call_df = pd.DataFrame(index = opa_atm_open_df.loc[opa_atm_open_df["call_put"] == "call", "date"].unique())

call_df = call_df.merge(
    opa_atm_open_df.loc[opa_atm_open_df["call_put"] == "call", ["date", "strike_price", "open"]]
               .rename(columns = {"open": "prev_night_open"}),
    left_index = True,
    right_on = "date",
    how = "left"
)
call_df["date"] = pd.to_datetime(call_df["date"])
call_df.sort_values("date", inplace = True)
call_df = call_df.merge(
    op_df.loc[op_df["call_put"] == "call", ["date", "strike_price", "open"]]
          .rename(columns = {"open": "day_open"}),
    on = ["date", "strike_price"],
    how = "left"
)
call_open_mask = (call_df["prev_night_open"] > 0) & (call_df["day_open"] > 0)
call_df["ret"] = np.where(
    call_open_mask,
    (call_df["day_open"] / call_df["prev_night_open"]) - 1,
    np.nan
)
call_df["gap_days"] = call_df["date"].diff().dt.days
call_df.reset_index(drop = True, inplace = True)

# get tx ret
merge_df = pd.merge(
    tx_df[["Timestamp", "Open"]].rename(columns = {"Open": "position_open"}),
    txa_df[["Timestamp", "Open"]].rename(columns = {"Open": "after_open"}),
    on = "Timestamp"
)
merge_df["Timestamp"] = pd.to_datetime(merge_df["Timestamp"])
merge_df["tx_ret"] = (merge_df["position_open"] / merge_df["after_open"]) - 1
call_df = call_df.merge(
    merge_df.rename(columns = {"Timestamp": "date"})[["date", "tx_ret"]],
    on = "date",
    how = "left"
)
call_df.dropna(subset = ["ret", "tx_ret"], inplace = True)

In [128]:
call_df.sort_values(["gap_days", "date"], ignore_index = True, inplace = True)

mean = call_df["ret"].mean()
call_df["demeaned_ret"] = call_df["ret"] - mean
call_df["cum_demeaned_ret"] = call_df["demeaned_ret"].cumsum()

mean = call_df["tx_ret"].mean()
call_df["demeaned_tx_ret"] = call_df["tx_ret"] - mean
call_df["cum_demeaned_tx_ret"] = call_df["demeaned_tx_ret"].cumsum()

call_df[["norm_cum_demeaned_ret", "norm_cum_demeaned_tx_ret"]] = pd.concat([
    call_df["cum_demeaned_ret"] / call_df["cum_demeaned_ret"].std(),
    call_df["cum_demeaned_tx_ret"] / call_df["cum_demeaned_tx_ret"].std()
], axis = 1)

plot(call_df.loc[call_df["gap_days"] < 2], ["norm_cum_demeaned_ret", "norm_cum_demeaned_tx_ret"], ry = "gap_days")
plot(call_df.loc[call_df["gap_days"] >= 2], ["norm_cum_demeaned_ret", "norm_cum_demeaned_tx_ret"], ry = "gap_days")

#### point

In [109]:
call_df = pd.DataFrame(index = opa_atm_open_df.loc[opa_atm_open_df["call_put"] == "call", "date"].unique())

call_df = call_df.merge(
    opa_atm_open_df.loc[opa_atm_open_df["call_put"] == "call", ["date", "strike_price", "open"]]
               .rename(columns = {"open": "prev_night_open"}),
    left_index = True,
    right_on = "date",
    how = "left"
)
call_df["date"] = pd.to_datetime(call_df["date"])
call_df.sort_values("date", inplace = True)
call_df = call_df.merge(
    op_df.loc[op_df["call_put"] == "call", ["date", "strike_price", "open"]]
          .rename(columns = {"open": "day_open"}),
    on = ["date", "strike_price"],
    how = "left"
)
call_mask = (call_df["prev_night_open"] > 0) & (call_df["day_open"] > 0)
call_df["ret"] = np.where(
    call_mask,
    (call_df["day_open"] - call_df["prev_night_open"]),
    np.nan
)
call_df["gap_days"] = call_df["date"].diff().dt.days
call_df.reset_index(drop = True, inplace = True)

# get tx ret
merge_df = pd.merge(
    tx_df[["Timestamp", "Open"]].rename(columns = {"Open": "position_open"}),
    txa_df[["Timestamp", "Open"]].rename(columns = {"Open": "after_open"}),
    on = "Timestamp"
)
merge_df["Timestamp"] = pd.to_datetime(merge_df["Timestamp"])
merge_df["tx_ret"] = (merge_df["position_open"] - merge_df["after_open"])
call_df = call_df.merge(
    merge_df.rename(columns = {"Timestamp": "date"})[["date", "tx_ret"]],
    on = "date",
    how = "left"
)
call_df.dropna(subset = ["ret", "tx_ret"], inplace = True)

In [110]:
call_df.sort_values(["gap_days", "date"], ignore_index = True, inplace = True)
mask = call_df["gap_days"] >= 2

# cum open_to_open ret
call_df["cum_ret"] = call_df["ret"].cumsum()
call_df.loc[mask, "cum_ret"] = call_df.loc[mask, "ret"].cumsum()
mean = call_df["ret"].mean()
call_df["demeaned_ret"] = call_df["ret"] - mean
call_df["cum_demeaned_ret"] = call_df["demeaned_ret"].cumsum()
call_df.loc[mask, "cum_demeaned_ret"] = call_df.loc[mask, "demeaned_ret"].cumsum()

# cum tx ret
call_df["cum_tx_ret"] = call_df["tx_ret"].cumsum()
call_df.loc[mask, "cum_tx_ret"] = call_df.loc[mask, "tx_ret"].cumsum()
mean = call_df["tx_ret"].mean()
call_df["demeaned_tx_ret"] = call_df["tx_ret"] - mean
call_df["cum_demeaned_tx_ret"] = call_df["demeaned_tx_ret"].cumsum()
call_df.loc[mask, "cum_demeaned_tx_ret"] = call_df.loc[mask, "demeaned_tx_ret"].cumsum()

plot(call_df.loc[call_df["gap_days"] < 2], ["cum_ret", "cum_tx_ret"], ry = "gap_days")
plot(call_df.loc[call_df["gap_days"] >= 2], ["cum_ret", "cum_tx_ret"], ry = "gap_days")

## 放假前 日盤 open - 日盤 close

### put

#### ret

In [125]:
"""get date gap days"""
put_df = op_atm_open_df.loc[op_atm_open_df["call_put"] == "put"].copy().reset_index(drop = True)

put_df["date"] = pd.to_datetime(put_df["date"])
put_df.sort_values("date", inplace = True)
put_df["gap_days"] = put_df["date"].diff().dt.days
put_df["gap_days"] = put_df["gap_days"].shift(-1)
put_df["daily_ret"] = (put_df["close"] / put_df["open"]) - 1

tx_day_ret = tx_df[["Timestamp", "Open", "Close"]].copy()
tx_day_ret["Timestamp"] = pd.to_datetime(tx_day_ret["Timestamp"])
tx_day_ret["tx_ret"] = (tx_day_ret["Close"] / tx_day_ret["Open"]) - 1
put_df = put_df.merge(
    tx_day_ret.rename(columns = {"Timestamp": "date"})[["date", "tx_ret"]],
    on = "date",
    how = "left"
)
put_df.dropna(subset = ["daily_ret", "tx_ret"], inplace = True)

In [126]:
put_df.sort_values(["gap_days", "date"], ignore_index = True, inplace = True)

mean = put_df["daily_ret"].mean()
put_df["demeaned_daily_ret"] = put_df["daily_ret"] - mean
put_df["cum_demeaned_daily_ret"] = put_df["demeaned_daily_ret"].cumsum()

mean = put_df["tx_ret"].mean()
put_df["demeaned_tx_ret"] = put_df["tx_ret"] - mean
put_df["cum_demeaned_tx_ret"] = put_df["demeaned_tx_ret"].cumsum()

put_df[["norm_cum_demeaned_daily_ret", "norm_cum_demeaned_tx_ret"]] = pd.concat([
    put_df["cum_demeaned_daily_ret"] / put_df["cum_demeaned_daily_ret"].std(),
    put_df["cum_demeaned_tx_ret"] / put_df["cum_demeaned_tx_ret"].std()
], axis = 1)

plot(put_df.loc[put_df["gap_days"] < 2], ["norm_cum_demeaned_daily_ret", "norm_cum_demeaned_tx_ret"], "index", "gap_days")
plot(put_df.loc[put_df["gap_days"] >= 2], ["norm_cum_demeaned_daily_ret", "norm_cum_demeaned_tx_ret"], "index", "gap_days")

#### point

In [3]:
"""get date gap days"""
put_df = op_atm_open_df.loc[op_atm_open_df["call_put"] == "put"].copy().reset_index(drop = True)

put_df["date"] = pd.to_datetime(put_df["date"])
put_df.sort_values("date", inplace = True)
put_df["gap_days"] = put_df["date"].diff().dt.days
put_df["gap_days"] = put_df["gap_days"].shift(-1)
put_df["ret"] = (put_df["close"] - put_df["open"])

tx_day_ret = tx_df[["Timestamp", "Open", "Close"]].copy()
tx_day_ret["Timestamp"] = pd.to_datetime(tx_day_ret["Timestamp"])
tx_day_ret["tx_ret"] = (tx_day_ret["Close"] - tx_day_ret["Open"])
put_df = put_df.merge(
    tx_day_ret.rename(columns = {"Timestamp": "date"})[["date", "tx_ret"]],
    on = "date",
    how = "left"
)
put_df.dropna(subset = ["ret", "tx_ret"], inplace = True)

In [5]:
put_df.sort_values(["gap_days", "date"], ignore_index = True, inplace = True)
mask = put_df["gap_days"] >= 2

# cum daily ret
put_df["cum_ret"] = put_df["ret"].cumsum()
put_df.loc[mask, "cum_ret"] = put_df.loc[mask, "ret"].cumsum()
mean = put_df["ret"].mean()
put_df["demeaned_ret"] = put_df["ret"] - mean
put_df["cum_demeaned_ret"] = put_df["demeaned_ret"].cumsum()

# cum tx ret
put_df["cum_tx_ret"] = put_df["tx_ret"].cumsum()
put_df.loc[mask, "cum_tx_ret"] = put_df.loc[mask, "tx_ret"].cumsum()
mean = put_df["tx_ret"].mean()
put_df["demeaned_tx_ret"] = put_df["tx_ret"] - mean
put_df["cum_demeaned_tx_ret"] = put_df["demeaned_tx_ret"].cumsum()

plot(put_df.loc[put_df["gap_days"] < 2], ["cum_ret", "cum_tx_ret"], ry = "gap_days")
plot(put_df.loc[put_df["gap_days"] >= 2], ["cum_ret", "cum_tx_ret"], ry = "gap_days")

### call

#### ret

In [9]:
"""get date gap days"""
call_df = op_atm_open_df.loc[op_atm_open_df["call_put"] == "call"].copy().reset_index(drop = True)

call_df["date"] = pd.to_datetime(call_df["date"])
call_df.sort_values("date", inplace = True)
call_df["gap_days"] = call_df["date"].diff().dt.days
call_df["gap_days"] = call_df["gap_days"].shift(-1)
call_df["daily_ret"] = (call_df["close"] / call_df["open"]) - 1

tx_day_ret = tx_df[["Timestamp", "Open", "Close"]].copy()
tx_day_ret["Timestamp"] = pd.to_datetime(tx_day_ret["Timestamp"])
tx_day_ret["tx_ret"] = (tx_day_ret["Close"] / tx_day_ret["Open"]) - 1
call_df = call_df.merge(
    tx_day_ret.rename(columns = {"Timestamp": "date"})[["date", "tx_ret"]],
    on = "date",
    how = "left"
)
call_df.dropna(subset = ["daily_ret", "tx_ret"], inplace = True)

In [10]:
call_df.sort_values(["gap_days", "date"], ignore_index = True, inplace = True)
mean = call_df["daily_ret"].mean()
call_df["demeaned_daily_ret"] = call_df["daily_ret"] - mean
call_df["cum_demeaned_daily_ret"] = call_df["demeaned_daily_ret"].cumsum()

mean = call_df["tx_ret"].mean()
call_df["demeaned_tx_ret"] = call_df["tx_ret"] - mean
call_df["cum_demeaned_tx_ret"] = call_df["demeaned_tx_ret"].cumsum()

call_df[["norm_cum_demeaned_daily_ret", "norm_cum_demeaned_tx_ret"]] = pd.concat([
    call_df["cum_demeaned_daily_ret"] / call_df["cum_demeaned_daily_ret"].std(),
    call_df["cum_demeaned_tx_ret"] / call_df["cum_demeaned_tx_ret"].std()
], axis = 1)

plot(call_df.loc[call_df["gap_days"] < 2], ["norm_cum_demeaned_daily_ret", "norm_cum_demeaned_tx_ret"], "index", "gap_days")
plot(call_df.loc[call_df["gap_days"] >= 2], ["norm_cum_demeaned_daily_ret", "norm_cum_demeaned_tx_ret"], "index", "gap_days")

#### point

In [11]:
call_df = op_atm_open_df.loc[op_atm_open_df["call_put"] == "call"].copy().reset_index(drop = True)

call_df["date"] = pd.to_datetime(call_df["date"])
call_df.sort_values("date", inplace = True)
call_df["gap_days"] = call_df["date"].diff().dt.days
call_df["gap_days"] = call_df["gap_days"].shift(-1)
call_df["daily_ret"] = (call_df["close"] - call_df["open"])

tx_day_ret = tx_df[["Timestamp", "Open", "Close"]].copy()
tx_day_ret["Timestamp"] = pd.to_datetime(tx_day_ret["Timestamp"])
tx_day_ret["tx_ret"] = (tx_day_ret["Close"] - tx_day_ret["Open"])
call_df = call_df.merge(
    tx_day_ret.rename(columns = {"Timestamp": "date"})[["date", "tx_ret"]],
    on = "date",
    how = "left"
)
call_df.dropna(subset = ["daily_ret", "tx_ret"], inplace = True)

In [12]:
call_df.sort_values(["gap_days", "date"], ignore_index = True, inplace = True)
mask = call_df["gap_days"] >= 2

# cum daily ret
call_df["cum_daily_ret"] = call_df["daily_ret"].cumsum()
call_df.loc[mask, "cum_daily_ret"] = call_df.loc[mask, "daily_ret"].cumsum()
mean = call_df["daily_ret"].mean()
call_df["demeaned_daily_ret"] = call_df["daily_ret"] - mean
call_df["cum_demeaned_daily_ret"] = call_df["demeaned_daily_ret"].cumsum()

# cum tx ret
call_df["cum_tx_ret"] = call_df["tx_ret"].cumsum()
call_df.loc[mask, "cum_tx_ret"] = call_df.loc[mask, "tx_ret"].cumsum()
mean = call_df["tx_ret"].mean()
call_df["demeaned_tx_ret"] = call_df["tx_ret"] - mean
call_df["cum_demeaned_tx_ret"] = call_df["demeaned_tx_ret"].cumsum()

plot(call_df.loc[call_df["gap_days"] < 2], ["cum_daily_ret", "cum_tx_ret"], ry = "gap_days")
plot(call_df.loc[call_df["gap_days"] >= 2], ["cum_daily_ret", "cum_tx_ret"], ry = "gap_days")

### compare to index

In [17]:
tw_df = twii_df.copy()
tw_df["daily_ret"] = (tw_df["close"] - tw_df["open"])

put_df.set_index("date", inplace = True)
call_df.set_index("date", inplace = True)
put_df.index, call_df.index, tw_df.index = pd.to_datetime(put_df.index), pd.to_datetime(call_df.index), pd.to_datetime(tw_df.index)

tw_df = tw_df.reindex(put_df.index)
tw_df["cum_daily_ret"] = tw_df["daily_ret"].cumsum()

put_df.reset_index(inplace = True)
call_df.reset_index(inplace = True)
tw_df.reset_index(inplace = True)

ret_df = pd.DataFrame({
    "cum_tw_ret": tw_df["cum_daily_ret"],
    "cum_put_ret": put_df["cum_ret"],
    "cum_call_ret": call_df["cum_daily_ret"]
})

ret_df_norm = ret_df / ret_df.std()
plot_df_columns(ret_df_norm, ["cum_tw_ret", "cum_put_ret", "cum_call_ret"])


## 放假前 夜盤 open - 夜盤 close

### put

#### ret

In [23]:
put_df = (
    opa_atm_open_df.loc[opa_atm_open_df["call_put"] == "put", ["date", "strike_price", "open", "close"]]
    .copy()
)
put_df["date"] = pd.to_datetime(put_df["date"])
put_df.sort_values("date", inplace = True)
put_mask = (put_df["open"] > 0) & (put_df["close"] > 0)
put_df["daily_ret"] = np.where(
    put_mask,
    (put_df["close"] / put_df["open"]) - 1,
    np.nan
)
put_df["gap_days"] = put_df["date"].diff().dt.days
put_df.reset_index(drop = True, inplace = True)

txa_day_ret = txa_df[["Timestamp", "Open", "Close"]].copy()
txa_day_ret["Timestamp"] = pd.to_datetime(txa_day_ret["Timestamp"])
txa_day_ret["tx_ret"] = (txa_day_ret["Close"] / txa_day_ret["Open"]) - 1
put_df = put_df.merge(
    txa_day_ret.rename(columns = {"Timestamp": "date"})[["date", "tx_ret"]],
    on = "date",
    how = "left"
)
put_df.dropna(subset = ["daily_ret", "tx_ret"], inplace = True)

In [24]:
put_df.sort_values(["gap_days", "date"], ignore_index = True, inplace = True)
mean = put_df["daily_ret"].mean()
put_df["demeaned_daily_ret"] = put_df["daily_ret"] - mean
put_df["cum_demeaned_daily_ret"] = put_df["demeaned_daily_ret"].cumsum()

mean = put_df["tx_ret"].mean()
put_df["demeaned_tx_ret"] = put_df["tx_ret"] - mean
put_df["cum_demeaned_tx_ret"] = put_df["demeaned_tx_ret"].cumsum()

put_df[["norm_cum_demeaned_daily_ret", "norm_cum_demeaned_tx_ret"]] = pd.concat([
    put_df["cum_demeaned_daily_ret"] / put_df["cum_demeaned_daily_ret"].std(),
    put_df["cum_demeaned_tx_ret"] / put_df["cum_demeaned_tx_ret"].std()
], axis = 1)

plot(put_df.loc[put_df["gap_days"] < 2], ["norm_cum_demeaned_daily_ret", "norm_cum_demeaned_tx_ret"], ry = "gap_days")
plot(put_df.loc[put_df["gap_days"] >= 2], ["norm_cum_demeaned_daily_ret", "norm_cum_demeaned_tx_ret"], ry = "gap_days")

#### point

In [26]:
put_df = (
    opa_atm_open_df.loc[opa_atm_open_df["call_put"] == "put", ["date", "strike_price", "open", "close"]]
    .copy()
)
put_df["date"] = pd.to_datetime(put_df["date"])
put_df.sort_values("date", inplace = True)
put_mask = (put_df["open"] > 0) & (put_df["close"] > 0)
put_df["daily_ret"] = np.where(
    put_mask,
    (put_df["close"] - put_df["open"]),
    np.nan
)
put_df["gap_days"] = put_df["date"].diff().dt.days
put_df.reset_index(drop = True, inplace = True)

txa_day_ret = txa_df[["Timestamp", "Open", "Close"]].copy()
txa_day_ret["Timestamp"] = pd.to_datetime(txa_day_ret["Timestamp"])
txa_day_ret["tx_ret"] = (txa_day_ret["Close"] - txa_day_ret["Open"])
put_df = put_df.merge(
    txa_day_ret.rename(columns = {"Timestamp": "date"})[["date", "tx_ret"]],
    on = "date",
    how = "left"
)
put_df.dropna(subset = ["daily_ret", "tx_ret"], inplace = True)

In [27]:
put_df.sort_values(["gap_days", "date"], ignore_index = True, inplace = True)
mask = put_df["gap_days"] >= 2

# cum daily ret
put_df["cum_daily_ret"] = put_df["daily_ret"].cumsum()
put_df.loc[mask, "cum_daily_ret"] = put_df.loc[mask, "daily_ret"].cumsum()
mean = put_df["daily_ret"].mean()
put_df["demeaned_daily_ret"] = put_df["daily_ret"] - mean
put_df["cum_demeaned_daily_ret"] = put_df["demeaned_daily_ret"].cumsum()

# cum tx ret
put_df["cum_tx_ret"] = put_df["tx_ret"].cumsum()
put_df.loc[mask, "cum_tx_ret"] = put_df.loc[mask, "tx_ret"].cumsum()
mean = put_df["tx_ret"].mean()
put_df["demeaned_tx_ret"] = put_df["tx_ret"] - mean
put_df["cum_demeaned_tx_ret"] = put_df["demeaned_tx_ret"].cumsum()

plot(put_df.loc[put_df["gap_days"] < 2], ["cum_daily_ret", "cum_tx_ret"], ry = "gap_days")
plot(put_df.loc[put_df["gap_days"] >= 2], ["cum_daily_ret", "cum_tx_ret"], ry = "gap_days")

### call

#### ret

In [28]:
call_df = (
    opa_atm_open_df.loc[opa_atm_open_df["call_put"] == "call", ["date", "strike_price", "open", "close"]]
    .copy()
)
call_df["date"] = pd.to_datetime(call_df["date"])
call_df.sort_values("date", inplace = True)
call_mask = (call_df["open"] > 0) & (call_df["close"] > 0)
call_df["daily_ret"] = np.where(
    call_mask,
    (call_df["close"] / call_df["open"]) - 1,
    np.nan
)
call_df["gap_days"] = call_df["date"].diff().dt.days
call_df.reset_index(drop = True, inplace = True)

txa_day_ret = txa_df[["Timestamp", "Open", "Close"]].copy()
txa_day_ret["Timestamp"] = pd.to_datetime(txa_day_ret["Timestamp"])
txa_day_ret["tx_ret"] = (txa_day_ret["Close"] / txa_day_ret["Open"]) - 1
call_df = call_df.merge(
    txa_day_ret.rename(columns = {"Timestamp": "date"})[["date", "tx_ret"]],
    on = "date",
    how = "left"
)
call_df.dropna(subset = ["daily_ret", "tx_ret"], inplace = True)

In [29]:
call_df.sort_values(["gap_days", "date"], ignore_index = True, inplace = True)

mean = call_df["daily_ret"].mean()
call_df["demeaned_daily_ret"] = call_df["daily_ret"] - mean
call_df["cum_demeaned_daily_ret"] = call_df["demeaned_daily_ret"].cumsum()

mean = call_df["tx_ret"].mean()
call_df["demeaned_tx_ret"] = call_df["tx_ret"] - mean
call_df["cum_demeaned_tx_ret"] = call_df["demeaned_tx_ret"].cumsum()

call_df[["norm_cum_demeaned_daily_ret", "norm_cum_demeaned_tx_ret"]] = pd.concat([
    call_df["cum_demeaned_daily_ret"] / call_df["cum_demeaned_daily_ret"].std(),
    call_df["cum_demeaned_tx_ret"] / call_df["cum_demeaned_tx_ret"].std()
], axis = 1)

plot(call_df.loc[call_df["gap_days"] < 2], ["norm_cum_demeaned_daily_ret", "norm_cum_demeaned_tx_ret"], ry = "gap_days")
plot(call_df.loc[call_df["gap_days"] >= 2], ["norm_cum_demeaned_daily_ret", "norm_cum_demeaned_tx_ret"], ry = "gap_days")

#### point

In [35]:
call_df = (
    opa_atm_open_df.loc[opa_atm_open_df["call_put"] == "call", ["date", "strike_price", "open", "close"]]
    .copy()
)
call_df["date"] = pd.to_datetime(call_df["date"])
call_df.sort_values("date", inplace = True)
call_mask = (call_df["open"] > 0) & (call_df["close"] > 0)
call_df["daily_ret"] = np.where(
    call_mask,
    (call_df["close"] - call_df["open"]),
    np.nan
)
call_df["gap_days"] = call_df["date"].diff().dt.days
call_df.reset_index(drop = True, inplace = True)

txa_day_ret = txa_df[["Timestamp", "Open", "Close"]].copy()
txa_day_ret["Timestamp"] = pd.to_datetime(txa_day_ret["Timestamp"])
txa_day_ret["tx_ret"] = (txa_day_ret["Close"] - txa_day_ret["Open"])
call_df = call_df.merge(
    txa_day_ret.rename(columns = {"Timestamp": "date"})[["date", "tx_ret"]],
    on = "date",
    how = "left"
)
call_df.dropna(subset = ["daily_ret", "tx_ret"], inplace = True)

In [36]:
call_df.sort_values(["gap_days", "date"], ignore_index = True, inplace = True)
mask = call_df["gap_days"] >= 2

# cum daily ret
call_df["cum_daily_ret"] = call_df["daily_ret"].cumsum()
call_df.loc[mask, "cum_daily_ret"] = call_df.loc[mask, "daily_ret"].cumsum()
mean = call_df["daily_ret"].mean()
call_df["demeaned_daily_ret"] = call_df["daily_ret"] - mean
call_df["cum_demeaned_daily_ret"] = call_df["demeaned_daily_ret"].cumsum()

# cum tx ret
call_df["cum_tx_ret"] = call_df["tx_ret"].cumsum()
call_df.loc[mask, "cum_tx_ret"] = call_df.loc[mask, "tx_ret"].cumsum()
mean = call_df["tx_ret"].mean()
call_df["demeaned_tx_ret"] = call_df["tx_ret"] - mean
call_df["cum_demeaned_tx_ret"] = call_df["demeaned_tx_ret"].cumsum()

plot(call_df.loc[call_df["gap_days"] < 2], ["cum_daily_ret", "cum_tx_ret"], ry = "gap_days")
plot(call_df.loc[call_df["gap_days"] >= 2], ["cum_daily_ret", "cum_tx_ret"], ry = "gap_days")

# backtest

## short put

### get df

In [50]:
# 日盤 ～ 夜盤
## merge opa, tx, txa 資料
put_df = op_atm_open_df.loc[op_atm_open_df["call_put"] == "put"].copy()
put_df["date"] = pd.to_datetime(put_df["date"])
put_df.rename(columns = {
    "open": "open_op",
    "close": "close_op"
}, inplace = True)
put_df["gap_days"] = put_df["date"].diff().dt.days.shift(-1)
put_df = put_df.merge(
    opa_df.loc[opa_df["call_put"] == "put", ["date", "strike_price", "open", "close"]].rename(columns = {"open": "open_opa", "close": "close_opa"}),
    left_on = ["date", "strike_price"], right_on = ["date", "strike_price"],
    how = "left"
)
put_df = put_df.merge(
    tx_df[["Timestamp", "Open", "Close"]].rename(columns = {"Timestamp": "date", "Open": "open_tx", "Close": "close_tx"}),
    on = ["date"],
    how = "left"
)
put_df = put_df.merge(
    txa_df[["Timestamp", "Open", "Close"]].rename(columns = {"Timestamp": "date", "Open": "open_txa", "Close": "close_txa"}),
    on = ["date"],
    how = "left"
)

# 夜盤 ～ 日盤
## merge op, tx, txa 資料
put_a_df = opa_atm_open_df.loc[opa_atm_open_df["call_put"] == "put"].copy()
put_a_df["date"] = pd.to_datetime(put_a_df["date"])
put_a_df.rename(columns = {
    "open": "open_opa",
    "close": "close_opa"
}, inplace = True)
put_a_df["gap_days"] = put_a_df["date"].diff().dt.days
put_a_df = put_a_df.merge(
    op_df.loc[op_df["call_put"] == "put", ["date", "strike_price", "open", "close"]].rename(columns = {"open": "open_op", "close": "close_op"}),
    on = ["date", "strike_price"],
    how = "left"
)
put_a_df = put_a_df.merge(
    tx_df[["Timestamp", "Open", "Close"]].rename(columns = {"Timestamp": "date", "Open": "open_tx", "Close": "Close_tx"}),
    on = ["date"],
    how = "left"
)
put_a_df = put_a_df.merge(
    txa_df[["Timestamp", "Open", "Close"]].rename(columns = {"Timestamp": "date", "Open": "open_txa", "Close": "close_txa"}),
    on = ["date"],
    how = "left"
)

### 不分假日

In [65]:
# 日盤 oepn ~ close
bt_df = put_df.copy()
bt_df["capitalization"] = bt_df["open_op"] * 50
bt_df["daily_ret"] = np.where(
    bt_df["capitalization"] > 0,
    (bt_df["close_op"] - bt_df["open_op"]) / bt_df["capitalization"],
    np.nan
)
bt_df["tx_ret"] = (bt_df["close_tx"] / bt_df["open_tx"]) - 1
bt_df["cum_daily_ret"], bt_df["cum_tx_ret"] = bt_df["daily_ret"].cumsum(), bt_df["tx_ret"].cumsum()

print("----- 日盤 open ~ close -----")
plot(bt_df, ly = "cum_daily_ret", x = "date", ry = "cum_tx_ret", ry_dashed = False)
perf = summarize_performance(bt_df["daily_ret"])
for key, value in perf.to_dict().items():
    print(f"{key}, {value}")
print("\n")


# 夜盤 open ~ 日盤 open
bt_df = put_a_df.copy()
bt_df["capitalization"] = bt_df["open_opa"] * 50
bt_df["daily_ret"] = np.where(
    bt_df["capitalization"] > 0,
    (bt_df["open_op"] - bt_df["open_opa"]) / bt_df["capitalization"],
    np.nan
)
bt_df["tx_ret"] = (bt_df["open_tx"] / bt_df["open_txa"]) - 1
bt_df["cum_daily_ret"], bt_df["cum_tx_ret"] = bt_df["daily_ret"].cumsum(), bt_df["tx_ret"].cumsum()

print("----- 夜盤 open ~ 日盤 open -----")
plot(bt_df, ly = "cum_daily_ret", x = "date", ry = "cum_tx_ret", ry_dashed = False)
perf = summarize_performance(bt_df["daily_ret"])
for key, value in perf.to_dict().items():
    print(f"{key}, {value}")
print("\n")


# 夜盤 open ~ close
bt_df = put_a_df.copy()
bt_df["capitalization"] = bt_df["open_opa"] * 50
bt_df["daily_ret"] = np.where(
    bt_df["capitalization"] > 0,
    (bt_df["close_opa"] - bt_df["open_opa"]) / bt_df["capitalization"],
    np.nan
)
bt_df["tx_ret"] = (bt_df["close_txa"] / bt_df["open_txa"]) - 1
bt_df["cum_daily_ret"], bt_df["cum_tx_ret"] = bt_df["daily_ret"].cumsum(), bt_df["tx_ret"].cumsum()

print("----- 夜盤 open ~ close -----")
plot(bt_df, ly = "cum_daily_ret", x = "date", ry = "cum_tx_ret", ry_dashed = False)
perf = summarize_performance(bt_df["daily_ret"])
for key, value in perf.to_dict().items():
    print(f"{key}, {value}")
print("\n")


# 夜盤 close ~ 日盤 open
bt_df = put_a_df.copy()
bt_df["capitalization"] = bt_df["close_opa"] * 50
bt_df["daily_ret"] = np.where(
    bt_df["capitalization"] > 0,
    (bt_df["open_op"] - bt_df["close_opa"]) / bt_df["capitalization"],
    np.nan
)
bt_df["tx_ret"] = (bt_df["open_tx"] / bt_df["close_txa"]) - 1
bt_df["cum_daily_ret"], bt_df["cum_tx_ret"] = bt_df["daily_ret"].cumsum(), bt_df["tx_ret"].cumsum()

print("----- 夜盤 close ~ 日盤 open -----")
plot(bt_df, ly = "cum_daily_ret", x = "date", ry = "cum_tx_ret", ry_dashed = False)
perf = summarize_performance(bt_df["daily_ret"])
for key, value in perf.to_dict().items():
    print(f"{key}, {value}")
print("\n")

----- 日盤 open ~ close -----


total_return, -1.05300132124535
cagr, -0.19049270133081708
annual_vol, 0.29630671549490645
sharpe, -0.6428902598871125
sortino, -1.0398053858430232
calmar, -0.1758533613944669
max_drawdown, -1.0832474274035162
max_drawdown_duration, 1379
win_rate, 0.36611629576453697
even_rate, 0.010050251256281407
avg_gain, 0.014287734018202696
avg_loss, -0.00959694553570624
mean, -0.0007559234179794329
kelly_fraction, -0.06380447809517685
observations, 1393


----- 夜盤 open ~ 日盤 open -----


total_return, -0.38466753362158657
cagr, -0.0695381768096412
annual_vol, 0.23908441977828093
sharpe, -0.2908519797071203
sortino, -0.4740234408846635
calmar, -0.11176177598267811
max_drawdown, -0.6221999981498045
max_drawdown_duration, 710
win_rate, 0.3500717360114778
even_rate, 0.011477761836441894
avg_gain, 0.013527002591143802
avg_loss, -0.007849263818089619
mean, -0.0002759451460700047
kelly_fraction, -0.015375382298883534
observations, 1394


----- 夜盤 open ~ close -----


total_return, -1.5926946902375558
cagr, -0.28668504424276003
annual_vol, 0.19022686450166235
sharpe, -1.5070691776042746
sortino, -2.071063333591456
calmar, -0.1861501719088217
max_drawdown, -1.5400740235855457
max_drawdown_duration, 1354
win_rate, 0.35214285714285715
even_rate, 0.009285714285714286
avg_gain, 0.010091280148695972
avg_loss, -0.007346415887633858
mean, -0.001137639064455397
kelly_fraction, -0.142101173713123
observations, 1400


----- 夜盤 close ~ 日盤 open -----


total_return, 3.3612770032536137
cagr, 0.6076340063270521
annual_vol, 0.3743567896628328
sharpe, 1.6231414070900707
sortino, 8.395051411379292
calmar, 10.141228820856973
max_drawdown, -0.05991719712283394
max_drawdown_duration, 696
win_rate, 0.4375896700143472
even_rate, 0.040889526542324243
avg_gain, 0.00959450785901511
avg_loss, -0.0034269226832814347
mean, 0.0024112460568533815
kelly_fraction, 0.23671050607383481
observations, 1394




### 分假日

In [ ]:
# 日盤 oepn ~ close
for i in range(2):
    lt = ["week", "weekend"]
    

# 夜盤 open ~ 日盤 open
# 夜盤 open ~ close
# 夜盤 close ~ 日盤 open

week
weekend


In [7]:
bm_op_df = op_atm_open_df.loc[op_atm_open_df["call_put"] == "put"].copy()
bm_opa_df = opa_atm_open_df.loc[opa_atm_open_df["call_put"] == "put"].copy()
tx_df["daily_ret"], txa_df["daily_ret"] = (tx_df["Close"] / tx_df["Open"]) - 1, (txa_df["Close"] / txa_df["Open"]) - 1

for i in ["", "a"]:
    vars()[f"bm_op{i}_df"]["capitalization"] = vars()[f"bm_op{i}_df"]["open"] * 50
    vars()[f"bm_op{i}_df"]["gap_days"] = vars()[f"bm_op{i}_df"]["date"].diff().dt.days
    vars()[f"bm_op{i}_df"]["daily_ret"] = (vars()[f"bm_op{i}_df"]["close"] - vars()[f"bm_op{i}_df"]["open"]) / vars()[f"bm_op{i}_df"]["capitalization"]
    vars()[f"bm_op{i}_df"] = vars()[f"bm_op{i}_df"].merge(
        vars()[f"tx{i}_df"][["Timestamp", "daily_ret"]].rename(columns = {"Timestamp": "date", "daily_ret": "tx_ret"}),
        left_on = "date",
        right_on = "date",
        how = "left"
    )
bm_op_df["gap_days"] = bm_op_df["gap_days"].shift(-1)

for i in ["", "a"]:    
    for j in ["week", "weekend"]:
        if i == "":
            if j == "week":
                print("----- 日盤 平日-----")
            elif j == "weekend":
                print("----- 日盤 假日 -----")
        elif i == "a":
            if j == "week":
                print("----- 夜盤 平日-----")
            elif j == "weekend":
                print("----- 夜盤 假日 -----")
    
        if j == "week":
            vars()[f"bm_op{i}_{j}_df"] = vars()[f"bm_op{i}_df"].loc[vars()[f"bm_op{i}_df"]["gap_days"] < 2].copy()
        elif j == "weekend":
            vars()[f"bm_op{i}_{j}_df"] = vars()[f"bm_op{i}_df"].loc[vars()[f"bm_op{i}_df"]["gap_days"] >= 2].copy()
            
        vars()[f"bm_op{i}_{j}_df"]["ret"] = - vars()[f"bm_op{i}_{j}_df"]["daily_ret"]
        vars()[f"bm_op{i}_{j}_df"]["cum_ret"] = vars()[f"bm_op{i}_{j}_df"]["ret"].cumsum()
        vars()[f"bm_op{i}_{j}_df"]["cum_tx_ret"] = vars()[f"bm_op{i}_{j}_df"]["tx_ret"].cumsum()
        
        plot(vars()[f"bm_op{i}_{j}_df"], x = "date", ly = ["cum_ret"], ry = "cum_tx_ret", ry_dashed = False)
        perf = summarize_performance(vars()[f"bm_op{i}_{j}_df"]["ret"])
        for key, value in perf.to_dict().items():
            print(f"{key}: {value}")
        print("\n")

----- 日盤 平日-----


total_return: 1.1224435982480618
cagr: 0.260216915141225
annual_vol: 0.31437785322821904
sharpe: 0.8277202495950741
sortino: 0.5744832088327466
calmar: 0.9605290991797607
max_drawdown: -0.2709099759324689
max_drawdown_duration: 145
hit_rate: 0.62557497700092
avg_gain: 0.01057382858115028
avg_loss: -0.015322625850843758
expectancy: 0.0010326068061159723
kelly_fraction: 0.0829924589869624
observations: 1087


----- 日盤 假日 -----


total_return: -0.06446666724661429
cagr: -0.05326426277425181
annual_vol: 0.22049647906223244
sharpe: -0.24156513972823404
sortino: -0.16348239917177929
calmar: -0.24242574463411803
max_drawdown: -0.2197137224622785
max_drawdown_duration: 225
hit_rate: 0.6196721311475409
avg_gain: 0.0060822340494525624
avg_loss: -0.01074344161586857
expectancy: -0.00021136612212004684
kelly_fraction: -0.05212547162177044
observations: 305


----- 夜盤 平日-----


total_return: 1.327368497759414
cagr: 0.3046419503054393
annual_vol: 0.19849071766761023
sharpe: 1.5347919231950604
sortino: 1.1020917923347677
calmar: 1.8779462726045375
max_drawdown: -0.16222080192045596
max_drawdown_duration: 189
hit_rate: 0.6411657559198543
avg_gain: 0.0075308821859602255
avg_loss: -0.010323045613393728
expectancy: 0.0012088966281961877
kelly_fraction: 0.14928948195651504
observations: 1098


----- 夜盤 假日 -----


total_return: 0.25259891975086907
cagr: 0.21147816537282063
annual_vol: 0.15665612588465092
sharpe: 1.3499514569161266
sortino: 1.0321785199199427
calmar: 1.6240191109406712
max_drawdown: -0.13021901278632575
max_drawdown_duration: 86
hit_rate: 0.627906976744186
avg_gain: 0.006630833184663483
avg_loss: -0.009265079186578972
expectancy: 0.0008391990689397644
kelly_fraction: 0.10799172186907076
observations: 301




## short call

### 不分假日

In [66]:
bm_op_df = op_atm_open_df.loc[op_atm_open_df["call_put"] == "call"].copy()
bm_opa_df = opa_atm_open_df.loc[opa_atm_open_df["call_put"] == "call"].copy()
tx_df["daily_ret"], txa_df["daily_ret"] = (tx_df["Close"] / tx_df["Open"]) - 1, (txa_df["Close"] / txa_df["Open"]) - 1

for i in ["", "a"]:
    vars()[f"bm_op{i}_df"]["capitalization"] = vars()[f"bm_op{i}_df"]["open"] * 50
    vars()[f"bm_op{i}_df"]["daily_ret"] = (vars()[f"bm_op{i}_df"]["close"] - vars()[f"bm_op{i}_df"]["open"]) / vars()[f"bm_op{i}_df"]["capitalization"]
    vars()[f"bm_op{i}_df"] = vars()[f"bm_op{i}_df"].merge(
        vars()[f"tx{i}_df"][["Timestamp", "daily_ret"]].rename(columns = {"Timestamp": "date", "daily_ret": "tx_ret"}),
        left_on = "date",
        right_on = "date",
        how = "left"
    )
    vars()[f"bm_op{i}_df"]["ret"] = - vars()[f"bm_op{i}_df"]["daily_ret"]
    vars()[f"bm_op{i}_df"]["cum_ret"] = vars()[f"bm_op{i}_df"]["ret"].cumsum()
    vars()[f"bm_op{i}_df"]["cum_tx_ret"] = vars()[f"bm_op{i}_df"]["tx_ret"].cumsum()
    if i == "":
        print("----- 日盤 -----")
    elif i == "a":
        print("----- 夜盤 -----")
    
    plot(vars()[f"bm_op{i}_df"], ["cum_ret"], ry = "cum_tx_ret", ry_dashed = False)
    perf = summarize_performance(vars()[f"bm_op{i}_{j}_df"]["ret"])
    for key, value in perf.to_dict().items():
        print(f"{key}: {value}")
    print("\n")

----- 日盤 -----


total_return: 0.28866815791200207
cagr: 0.23850615014368695
annual_vol: 0.12957746650197657
sharpe: 1.8406452648157678
sortino: 1.7982805758311344
calmar: 3.5036163501092146
max_drawdown: -0.06807427706411184
max_drawdown_duration: 56
win_rate: 0.5704918032786885
even_rate: 0.006557377049180328
avg_gain: 0.00649182467649046
avg_loss: -0.006518677021684792
mean: 0.0009464529767606626
kelly_fraction: 0.13920701785745085
observations: 305


----- 夜盤 -----


total_return: 0.18843739013824698
cagr: 0.15776153592969513
annual_vol: 0.1241310523821124
sharpe: 1.2709272410263475
sortino: 1.227896717312053
calmar: 1.37526171946265
max_drawdown: -0.11471382770061878
max_drawdown_duration: 143
win_rate: 0.5348837209302325
even_rate: 0.013289036544850499
avg_gain: 0.006298910629862808
avg_loss: -0.006071229568159302
mean: 0.0006260378409908537
kelly_fraction: 0.08657958199976643
observations: 301




### 分假日

In [34]:
bm_op_df = op_atm_open_df.loc[op_atm_open_df["call_put"] == "call"].copy()
bm_opa_df = opa_atm_open_df.loc[opa_atm_open_df["call_put"] == "call"].copy()
tx_df["daily_ret"], txa_df["daily_ret"] = (tx_df["Close"] / tx_df["Open"]) - 1, (txa_df["Close"] / txa_df["Open"]) - 1

for i in ["", "a"]:
    vars()[f"bm_op{i}_df"]["capitalization"] = vars()[f"bm_op{i}_df"]["open"] * 50
    vars()[f"bm_op{i}_df"]["gap_days"] = vars()[f"bm_op{i}_df"]["date"].diff().dt.days
    vars()[f"bm_op{i}_df"]["daily_ret"] = (vars()[f"bm_op{i}_df"]["close"] - vars()[f"bm_op{i}_df"]["open"]) / vars()[f"bm_op{i}_df"]["capitalization"]
    vars()[f"bm_op{i}_df"] = vars()[f"bm_op{i}_df"].merge(
        vars()[f"tx{i}_df"][["Timestamp", "daily_ret"]].rename(columns = {"Timestamp": "date", "daily_ret": "tx_ret"}),
        left_on = "date",
        right_on = "date",
        how = "left"
    )
bm_op_df["gap_days"] = bm_op_df["gap_days"].shift(-1)
for i in ["", "a"]:    
    for j in ["week", "weekend"]:
        if i == "":
            if j == "week":
                print("----- 日盤 平日-----")
            elif j == "weekend":
                print("----- 日盤 假日 -----")
        elif i == "a":
            if j == "week":
                print("----- 夜盤 平日-----")
            elif j == "weekend":
                print("----- 夜盤 假日 -----")

        if j == "week":
            vars()[f"bm_op{i}_{j}_df"] = vars()[f"bm_op{i}_df"].loc[vars()[f"bm_op{i}_df"]["gap_days"] < 2].copy()
        elif j == "weekend":
            vars()[f"bm_op{i}_{j}_df"] = vars()[f"bm_op{i}_df"].loc[vars()[f"bm_op{i}_df"]["gap_days"] >= 2].copy()
            
        vars()[f"bm_op{i}_{j}_df"]["ret"] = - vars()[f"bm_op{i}_{j}_df"]["daily_ret"]
        vars()[f"bm_op{i}_{j}_df"]["cum_ret"] = vars()[f"bm_op{i}_{j}_df"]["ret"].cumsum()
        vars()[f"bm_op{i}_{j}_df"]["cum_tx_ret"] = vars()[f"bm_op{i}_{j}_df"]["tx_ret"].cumsum()
        
        plot(vars()[f"bm_op{i}_{j}_df"], ["cum_ret"], ry = "cum_tx_ret", ry_dashed = False)
        perf = summarize_performance(vars()[f"bm_op{i}_{j}_df"]["ret"])
        for key, value in perf.to_dict().items():
            print(f"{key}: {value}")
        print("\n")

----- 日盤 平日-----


total_return: -1.5146284045067897
cagr: -0.35049252335694303
annual_vol: 0.2909922099295222
sharpe: -1.2044739047888318
sortino: -0.8849975656466248
calmar: -0.22909406975223298
max_drawdown: -1.529906573906533
max_drawdown_duration: 1079
hit_rate: 0.5583103764921947
avg_gain: 0.008893234196154658
avg_loss: -0.014572031148986992
expectancy: -0.001390843346654536
kelly_fraction: -0.09208358103405599
observations: 1089


----- 日盤 假日 -----


total_return: 0.28866815791200207
cagr: 0.23850615014368695
annual_vol: 0.12957746650197657
sharpe: 1.8406452648157678
sortino: 1.7982805758311344
calmar: 3.5036163501092146
max_drawdown: -0.06807427706411184
max_drawdown_duration: 56
hit_rate: 0.5704918032786885
avg_gain: 0.00649182467649046
avg_loss: -0.006518677021684792
expectancy: 0.0009464529767606626
kelly_fraction: 0.13920701785745085
observations: 305


----- 夜盤 平日-----


total_return: -0.44835851480546507
cagr: -0.10290195421764772
annual_vol: 0.15254850184636728
sharpe: -0.6745523749638724
sortino: -0.5870232380002697
calmar: -0.21697288729568048
max_drawdown: -0.47426180985192756
max_drawdown_duration: 1033
hit_rate: 0.5072859744990893
avg_gain: 0.006531833311191389
avg_loss: -0.007624234457349009
expectancy: -0.00040834108816526876
kelly_fraction: -0.04965703224988216
observations: 1098


----- 夜盤 假日 -----


total_return: 0.18843739013824698
cagr: 0.15776153592969513
annual_vol: 0.1241310523821124
sharpe: 1.2709272410263475
sortino: 1.227896717312053
calmar: 1.37526171946265
max_drawdown: -0.11471382770061878
max_drawdown_duration: 143
hit_rate: 0.5348837209302325
avg_gain: 0.006298910629862808
avg_loss: -0.006071229568159302
expectancy: 0.0006260378409908537
kelly_fraction: 0.08657958199976643
observations: 301


